In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')

In [3]:
trainingTable = pd.read_csv('trainingTable.csv')

In [4]:
trainingTable.head()

,PMCID,RCT,PMCFILE
0,PMC29099,False,PMC29099.nxml
1,PMC29105,False,PMC29105.nxml
2,PMC32159,False,PMC32159.nxml
3,PMC32179,False,PMC32179.nxml
4,PMC35278,False,PMC35278.nxml


In [5]:
dest = 'RCTData/'

In [6]:
data_X = []
data_Y = []

# Load stop-words
stop_words = set(stopwords.words('english'))
stop_words.update(('name','surname','given','first'))

# Initialize tokenizer
regexp_tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

# Initialize lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [7]:
# Tokenize a document to words
def tokenize(document, rebuild_document=True):
    words = []

    for sentence in sent_tokenize(document):
        tokens = [wordnet_lemmatizer.lemmatize(t.lower()) 
                  for t in regexp_tokenizer.tokenize(sentence) if t.lower() not in stop_words]
        words += tokens

    if rebuild_document:
        return ' '.join(words).strip()
    else:
        return words

In [8]:
# put all tokens into dataset data_X and assign the correspond labels
for i in tqdm.tqdm(range(len(trainingTable['PMCID']))):
    fileName = os.path.join(dest, trainingTable['PMCFILE'][i])
    with open(fileName, 'r',encoding='ISO-8859-1') as data_file:
        rawdata = data_file.read()
    soup = BeautifulSoup(rawdata)
    text = soup.get_text()
    data_X.append(tokenize(text))
    data_Y.append(trainingTable['RCT'][i])

100%|██████████| 19111/19111 [43:46<00:00,  7.28it/s]


In [9]:
p_file = 'data_X'

with open(p_file, 'wb') as fout:
    pickle.dump(data_X, fout)

In [10]:
# Create tagged documents
taggedDoc = []

for i, document in enumerate(data_X):
    taggedDoc.append(TaggedDocument(document.split(' '), [i]))

p_file = 'taggedDoc'
with open(p_file, 'wb') as fout:
    pickle.dump(taggedDoc, fout)

# Train Doc2Vec model
# https://arxiv.org/pdf/1405.4053v2.pdf
doc2vecSize = 32*32

doc2vecModel = Doc2Vec(documents=taggedDoc, size=doc2vecSize, window=3, 
                        min_count=2, iter=30, workers=multiprocessing.cpu_count())
doc2vecModel.save('doc2vecModel')
doc2vecModel.init_sims(replace=False)

# Vectorize documents and split corpus to training and testing
vectors = []

for document in data_X:
    vectors.append(doc2vecModel.infer_vector(document.split(' ')))

d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [11]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [12]:
p_file = 'RCT_Vectors'

with open(p_file, 'wb') as fout:
    pickle.dump(vectors, fout)

In [13]:
p_file = 'RCT_labels'

with open(p_file, 'wb') as fout:
    pickle.dump(data_Y, fout)